In [46]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

***DATA CLEANING ***

*SINCE WE'VE EXTRACTED THE DATA FROM THE WEBSITE, IT IS NOT CLEANED AND READY TO BE ANALYSED, THE REVIEW SECTION WILL NEED TO BE CLEANED FOR PUNCTUATIONS , SPELLING AND COLUMNS*

In [73]:
# IMPORT REQUIRED LIBRARIES
import pandas as pd
import os
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [48]:
#create dataframe from csv file
cwd = os.getcwd()
df = pd.read_csv(cwd+"/BA_reviews new.csv", index_col=0)
df.head()

,reviews,stars,date,country
0,Not Verified | They changed our Flights from ...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,18th April 2023,United States
1,Not Verified | At Copenhagen the most chaotic...,2,18th April 2023,United States
2,✅ Trip Verified | Worst experience of my life...,5,17th April 2023,United States
3,✅ Trip Verified | Due to code sharing with Ca...,1,17th April 2023,Hong Kong
4,✅ Trip Verified | LHR check in was quick at t...,3,16th April 2023,United Kingdom


we will also create a column which mentions if the user is verified or not

In [49]:
df['verified'] = df.reviews.str.contains("Trip Verified")
df['verified']

0       False
1       False
2        True
3        True
4        True
        ...  
3495    False
3496    False
3497    False
3498    False
3499    False
Name: verified, Length: 3500, dtype: bool

### **Cleaning**

We will extract the column of reviews into a separate dataframe and clean it for semantic analysis

In [50]:
#for lemmatization of words we will use nltk library
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

reviews_data = df.reviews.str.strip("✅ Trip Verified |")

#create an empty list to collect cleaned data corpus
corpus =[]

#loop through each review, remove punctuations, small case it, join it and add it to corpus
for rev in reviews_data:
    rev = re.sub('[^a-zA-Z]',' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in stop_words]
    rev = " ".join(rev)
    corpus.append(rev)

In [51]:
#add corpus to the original dataframe
df['corpus'] = corpus

In [52]:
df.head()

,reviews,stars,date,country,verified,corpus
0,Not Verified | They changed our Flights from ...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,18th April 2023,United States,False,verified changed flight brussels london heathr...
1,Not Verified | At Copenhagen the most chaotic...,2,18th April 2023,United States,False,verified copenhagen chaotic ticket counter ass...
2,✅ Trip Verified | Worst experience of my life...,5,17th April 2023,United States,True,worst experience life trying deal customer ser...
3,✅ Trip Verified | Due to code sharing with Ca...,1,17th April 2023,Hong Kong,True,due code sharing cathay pacific downgraded ba ...
4,✅ Trip Verified | LHR check in was quick at t...,3,16th April 2023,United Kingdom,True,lhr check quick first wing quickly security fi...


### cleaning Date and convert to date time format

In [53]:
df.dtypes

reviews     object
stars       object
date        object
country     object
verified      bool
corpus      object
dtype: object

In [54]:
#convert date to datetime format
df.date = pd.to_datetime(df.date)
df.date.head()

0   2023-04-18
1   2023-04-18
2   2023-04-17
3   2023-04-17
4   2023-04-16
Name: date, dtype: datetime64[ns]

** Cleaning ratings with stars **

In [55]:
#check the null values
df.stars.unique()

array(['\n\t\t\t\t\t\t\t\t\t\t\t\t\t5', '2', '5', '1', '3', '4', '9', '7',
       '10', '8', '6', 'None'], dtype=object)

In [59]:
#remove the \n and \t values from the ratings
df.stars = df.stars.str.strip("\n\t\t\t\t\t\t\t\t\t\t\t\t\t")

In [61]:
df.stars.value_counts()

1       774
2       394
3       389
8       350
10      312
9       300
7       300
5       261
4       235
6       181
None      4
Name: stars, dtype: int64

there are 4 rows having None value in the ratings now we drop that

In [62]:
#drop the row where the value of rating is None
df.drop(df[df.stars == "None"].index, axis=0, inplace=True)

In [63]:
#check the unique values again
df.stars.unique()

array(['5', '2', '1', '3', '4', '9', '7', '10', '8', '6'], dtype=object)

*check for null values*

In [66]:
df.isnull().value_counts()

reviews  stars  date   country  verified  corpus
False    False  False  False    False     False     3494
                       True     False     False        2
dtype: int64

In [67]:
df.country.isnull().value_counts()

False    3494
True        2
Name: country, dtype: int64

In [68]:
#drop the rows using index where the country value is null
df.drop(df[df.country.isnull() == True].index, axis=0, inplace=True)

In [69]:
df.shape

(3494, 6)

In [71]:
#resetting the index
df.reset_index(drop=True)

,reviews,stars,date,country,verified,corpus
0,Not Verified | They changed our Flights from ...,5,2023-04-18,United States,False,verified changed flight brussels london heathr...
1,Not Verified | At Copenhagen the most chaotic...,2,2023-04-18,United States,False,verified copenhagen chaotic ticket counter ass...
2,✅ Trip Verified | Worst experience of my life...,5,2023-04-17,United States,True,worst experience life trying deal customer ser...
3,✅ Trip Verified | Due to code sharing with Ca...,1,2023-04-17,Hong Kong,True,due code sharing cathay pacific downgraded ba ...
4,✅ Trip Verified | LHR check in was quick at t...,3,2023-04-16,United Kingdom,True,lhr check quick first wing quickly security fi...
...,...,...,...,...,...,...
3489,While different people have different experien...,5,2014-05-11,India,False,different people different experience wanted s...
3490,Once again a good flight from LHR to Warsaw in...,10,2014-05-11,United Kingdom,False,good flight lhr warsaw cabin crew efficient fr...
3491,LGW-MRU-LGW in business. Outbound flight good ...,6,2014-05-11,United Kingdom,False,lgw mru lgw business outbound flight good atte...
3492,LHR-FRA-LCY May 2014. LHR-FRA on a 767 row 18....,2,2014-05-11,United Kingdom,False,lhr fra lcy may lhr fra row checked via mobile...


Now our data is cleaned and ready for Data Analysis and Visualisation

In [72]:
#Export the cleaned data
df.to_csv(cwd + "/cleaned-BA-reviews.csv")